<a href="https://colab.research.google.com/github/ranabilal09/Rag-web-chatbot/blob/main/Rag_web_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade --quiet langchain-exa langchain langchain_community langchain-google-genai

In [6]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("Gemini_Api_Key")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("langchai_api_key")
os.environ["LANGCHAIN_PROJECT"] = "web-rag-search"
os.environ["EXA_API_KEY"] = userdata.get("exa_api_key")

In [7]:
from langchain_exa import ExaSearchRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda


retriever = ExaSearchRetriever(k=3, highlights=True)

document_prompt = PromptTemplate.from_template("""
<source>
    <url>{url}</url>
    <highlights>{highlights}</highlights>
</source>
""")

document_chain = RunnableLambda(
    lambda document: {
        "highlights": document.metadata["highlights"],
        "url": document.metadata["url"]
    }
) | document_prompt

retrieval_chain = retriever | document_chain.map() | (lambda docs: "\n".join([i.text for i in docs]))

print(retrieval_chain.invoke("best time to visit japan"))


<source>
    <url>https://www.responsibletravel.com/holidays/japan/travel-guide/best-time-to-visit-japan</url>
    <highlights>['July in Japan By July, the rains are easing and temperatures rising, and people head for beaches, beer gardens and higher ground where it’s cooler. Hike to the summit of the sacred Mount Fuji, an active volcano, which due to snow at other times of year can only be reached in July and August. Expect lots of company on the trails. Japan in July is festival season, events taking place across the country including one of the most significant, the Gion Matsuri in Kyoto, with parades, floats and delectable street food.']</highlights>
</source>


<source>
    <url>https://www.laviezine.com/394856/japan-top-destinations-2022/</url>
    <highlights>['japan travel update today,japan top destinations,japan is worth it to visit and have lots of unique culture i recomended for you,many best place you must visit like kobe,osaka,tokyo,hokaido jepan#japan #recommended #visi

In [8]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

generation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert research assistant. You use xml-formatted context to research people's questions."),
    ("human", """
Please answer the following query based on the provided context. Please cite your sources at the end of your response.:

Query: {query}
---
<context>
{context}
</context>
""")
])

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-8b",temperature=0.2)

chain = RunnableParallel({
    "query": RunnablePassthrough(),
    "context": retrieval_chain,
}) | generation_prompt | llm

In [12]:
chain.invoke("What is the economy rate of Pakistan ?")

AIMessage(content='Unfortunately, the provided context does not contain the specific economic rate for Pakistan.  While the sources mention economic indicators like the Consumer Price Index (CPI) and prize bonds, they do not provide a specific economic rate figure.  To get the economy rate of Pakistan, you would need to consult a more comprehensive source that explicitly states the rate.\n\n\n**Sources:**\n\n*  [https://www.khistocks.com/economics-indicators/economics-rates.html](https://www.khistocks.com/economics-indicators/economics-rates.html)\n*  [https://www.forex.pk/](https://www.forex.pk/)\n*  [https://countryeconomy.com/countries-cpi/pakistan](https://countryeconomy.com/countries-cpi/pakistan)\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',